### Scholarship Alert Project by Samuel Nnamani a.k.a SammystTheAnalyst

#### You can reach out to me for collaborations, consultations, and trainings on my Social Media handles
* YouTube: @SammystTheAnalyst
* Facebook Page: @SammystTheAnalyst
* X: @SammystDAnalyst
* LinkedIn: @SammystTheAnalyst

### STEP 1: Import Libraries

In [18]:
# import necessary libraries for web scraping the scholarship descriptions
from bs4 import BeautifulSoup
import requests
import re
import json
import os
import time

### STEP 2: Data Scraping

#### For this project, I scraped the scholarship title, link to apply, scholarship ad offer,  scholarship category, worth of award, scholarship amount, deadline, and grade level.

In [19]:
# Fetching the data from the website
data = []
for page in range(0, 51):
    url = "https://scholarships360.org/scholarships/top-scholarships-for-graduate-students/?sidebar_sort=relevant&current_page={page}&filter=all".format(page=page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    scholarships = soup.find_all("div", class_="re-scholarship-card-data-wrap")
    
# Using for loop to get the individual element tags for the scholarship description    
    for scholarship in scholarships:
        sch_info = {}
        title_1 = scholarship.find('h4').text.strip()
        sch_info['Title'] = str(title_1.split('\t')[0])                                       # Converts to string
        sch_info['Application_link'] = scholarship.find('a')["href"]
        org_offer_1 = scholarship.find('div').get_text()
        sch_info['Scholarship_Ad_Offer'] = str(org_offer_1.split('\n')[16])                   # Converts to string
        sch_class_1 = str(org_offer_1.split('\n')[17]) if org_offer_1 else "Not Available"    # Isolate only the class of scholarship
        sch_info['Scholarship_category'] = re.sub(r'\$\d{1,3}(,\d{3})*', "Not Available", sch_class_1)  # Replace any $ and , values with "Not Available"
        sch_info['Worth_of_award'] = scholarship.find('span', class_="re-scholarship-card-info-name").get_text()
        sch_info['Scholarship_amount'] = scholarship.find('span', class_="re-scholarship-card-info-value").get_text()
        deadline_1 = scholarship.find('div', class_="re-scholarship-card-mob_bottom").get_text()
        sch_info['Deadline'] = str(deadline_1.split('\n')[13])          # Converts to string
        sch_info['Grade_level'] = str(deadline_1.split('\n')[17])       # Converts to string
        data.append(sch_info)

### STEP 3: Setting UP the Telegram BOT configurations

##### Kindly note:
* 1. The bot_token displayed is not the original bot token issued to me by Telegram.
* 2. The chat_id displayed is not the original chat id issued to me by Telegram.

In [ ]:
# Setting up the Telegram BOT Credentials
bot_token = "7991611150:BBGn1-ycU9M1q5OkUbBPGYfZ_ugRcySqwlp"
chat_id = "2769920345"

# File to store sent scholarsships
sent_file = "scholarships.json"

# Function to load sent scholarships
def load_sent_file():
    if os.path.exists(sent_file):
        with open(sent_file, "r") as f:
            return set(json.load(f))
    return set()

# Function to save sent scholarships
def save_sent_file(sent_scholarships):
    with open(sent_file, "w") as f:
        json.dump(list(sent_scholarships), f)

# Function Telegram bot to receive the alert and send as a message.  
def send_telegram_message(message):
    url = f"https://api.telegram.org/bot7991611150:BBGn1-ycU9M1q5OkUbBPGYfZ_ugRcySqwlp/sendMessage"
    data = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
    
    # In the case of timeout, the function to check for errors and print
    try:
        response = requests.post(url, data=data)                 
        response.raise_for_status()                              # Check for HTTP errors
        time.sleep(1.5)                                          # Add delay for 1.5seconds
    except requests.exceptions.HTTPError as e:
        print(f"🚨 Telegram API Error: {e}")
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Request failed: {e}")
        
# Load already sent scholarships
sent_scholarships = load_sent_file()

# Loop through the scraped scholarship data
new_scholarships = []
for sch_info in data:
    # Use title as a unique identifier
    sch_id = sch_info["Title"]
    if sch_id not in sent_scholarships:
        message = f""" 🎓*New Scholarship Alert*
    📌 **Title:** {sch_info['Title']}
    🏢 **Offered By:** {sch_info['Scholarship_Ad_Offer']}
    💰 **Amount:** {sch_info['Scholarship_amount']}
    📅 **Deadline:** {sch_info['Deadline']}
    🎓 **Grade Level:** {sch_info['Grade_level']}
    🔗 [Apply Here]({sch_info['Application_link']})
    """

        send_telegram_message(message)
        new_scholarships.append(sch_id)

# Update the sent sccholarships file
sent_scholarships.update(new_scholarships)
# save_sent_scholarships(sent_scholarships)

### Additional Step but not compulsory
#### Sending the scraped data to Zapier for Automation process

In [3]:
# # Zapier Webhook URl
# zapier_webhook_url = "https://hooks.zapier.com/hooks/catch/22278614/2chovj9/"

# # Send data to Zapier Webhook
#     for sch_info in data:
#         response = requests.post(zapier_webhook_url, json=sch_info)
#         # print(f"Sent to Zapier: {sch_info}, Status: {response.status_code}")